<a href="https://colab.research.google.com/github/GauravSahani1417/PySpark-Basic-Implementation/blob/main/Conversion_of_Data_Frames_%7C_Spark_to_Pandas_%26_Pandas_to_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Setting up environment!
# Setting up the PySpark environment

# Install java 8
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Apache Spark binary: This link can change based on the version. Update this link with the latest version before using
!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz

# Unzip file
!tar -xf spark-3.0.2-bin-hadoop2.7.tgz

# Install findspark: Adds Pyspark to sys.path at runtime
!pip install -q findspark

# Install pyspark
!pip install pyspark

# Add environmental variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

# findspark will locate spark in the system
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,184 kB]


In [ ]:
#Initialize spark session!

from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("Hands-on PySpark on Google Colab") \
        .getOrCreate()

In [ ]:
spark

In [ ]:
!wget -q https://archive.ics.uci.edu/ml/machine-learning-databases/00603/in-vehicle-coupon-recommendation.csv -P sample_data/

In [ ]:
# We can set header='true' and inferSchema='true' to infer the schema while reading the data

filepath = "sample_data/in-vehicle-coupon-recommendation.csv"
spark_df = spark.read.format('csv').options(header='true', inferSchema='true').load(filepath)
spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+----------+------+---+-----------------+------------+------------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-------------+---+
|destination    |passanger|weather|temperature|time|coupon               |expiration|gender|age|maritalStatus    |has_children|education               |occupation|income         |car |Bar  |CoffeeHouse|CarryAway|RestaurantLessThan20|Restaurant20To50|toCoupon_GEQ5min|toCoupon_GEQ15min|toCoupon_GEQ25min|direction_same|direction_opp|Y  |
+---------------+---------+-------+-----------+----+---------------------+----------+------+---+-----------------+------------+------------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+--------

In [ ]:
spark_df.printSchema()

root
 |-- destination: string (nullable = true)
 |-- passanger: string (nullable = true)
 |-- weather: string (nullable = true)
 |-- temperature: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- coupon: string (nullable = true)
 |-- expiration: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- maritalStatus: string (nullable = true)
 |-- has_children: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- income: string (nullable = true)
 |-- car: string (nullable = true)
 |-- Bar: string (nullable = true)
 |-- CoffeeHouse: string (nullable = true)
 |-- CarryAway: string (nullable = true)
 |-- RestaurantLessThan20: string (nullable = true)
 |-- Restaurant20To50: string (nullable = true)
 |-- toCoupon_GEQ5min: integer (nullable = true)
 |-- toCoupon_GEQ15min: integer (nullable = true)
 |-- toCoupon_GEQ25min: integer (nullable = true)
 |-- direction_same: inte

Convert Spark DF to Pandas DF

* Once you convert into pandas, you can use all the python libraries for 
modeling, visualization, preprocessing, etc.

In [ ]:
pandas_df = spark_df.toPandas()
pandas_df.shape

(12684, 26)

In [ ]:
pandas_df.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,has_children,education,occupation,income,car,Bar,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,None,never,never,None,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,None,never,never,None,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,None,never,never,None,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,None,never,never,None,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,None,never,never,None,4~8,1~3,1,1,0,0,1,0


In [ ]:
# You can sample to sample the dataset and convert the data into pandas for further analysis
#eg. You have 1TB of data, you can just extract sample by keeping fraction=0.01, ie. 1% of whole data.

sample_pandas_df = spark_df.sample(withReplacement=False, fraction=0.01).toPandas()
sample_pandas_df.shape

(122, 26)

In [ ]:
pandas_df.shape

(12684, 26)

In [ ]:
sample_pandas_df["age"].value_counts()

26         32
21         24
50plus     18
36         17
31         17
41          9
46          3
below21     2
Name: age, dtype: int64

Convert Pandas DF to Spark DF

In [ ]:
# Select few columns
pandas_df_subset = pandas_df[["destination", "passanger", "weather", "temperature", "time", "coupon", "age", "car", "Y"]].copy()
pandas_df_subset.head(5)

,destination,passanger,weather,temperature,time,coupon,age,car,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),21,None,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,21,None,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,21,None,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,21,None,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,21,None,0


In [ ]:
pandas_df_subset.shape

(12684, 9)

In [ ]:
spark_df_from_pandas = spark.createDataFrame(pandas_df_subset)
spark_df_from_pandas.show(5)

+---------------+---------+-------+-----------+----+--------------------+---+----+---+
|    destination|passanger|weather|temperature|time|              coupon|age| car|  Y|
+---------------+---------+-------+-----------+----+--------------------+---+----+---+
|No Urgent Place|    Alone|  Sunny|         55| 2PM|     Restaurant(<20)| 21|null|  1|
|No Urgent Place|Friend(s)|  Sunny|         80|10AM|        Coffee House| 21|null|  0|
|No Urgent Place|Friend(s)|  Sunny|         80|10AM|Carry out & Take ...| 21|null|  1|
|No Urgent Place|Friend(s)|  Sunny|         80| 2PM|        Coffee House| 21|null|  0|
|No Urgent Place|Friend(s)|  Sunny|         80| 2PM|        Coffee House| 21|null|  0|
+---------------+---------+-------+-----------+----+--------------------+---+----+---+
only showing top 5 rows



In [ ]:
type(spark_df_from_pandas)

pyspark.sql.dataframe.DataFrame

In [ ]:
spark_df_from_pandas.printSchema()


root
 |-- destination: string (nullable = true)
 |-- passanger: string (nullable = true)
 |-- weather: string (nullable = true)
 |-- temperature: long (nullable = true)
 |-- time: string (nullable = true)
 |-- coupon: string (nullable = true)
 |-- age: string (nullable = true)
 |-- car: string (nullable = true)
 |-- Y: long (nullable = true)



Use schema for spark dataframe

* StructType objects define the schema of Spark DataFrames. StructType objects contain a list of StructField objects that define the name, type, and nullable flag for each column in a DataFrame.


In [ ]:
from pyspark.sql.types import *
schema = StructType([
      StructField("destination", StringType(), True),
      StructField("passanger", StringType(), True),
      StructField("weather", StringType(), True),
      StructField("temperature", IntegerType(), True),
      StructField("time", StringType(), True),
      StructField("coupon", StringType(), True),
      StructField("age", StringType(), True),
      StructField("car", StringType(), True),
      StructField("Y", IntegerType(), True)])

spark_df_from_pandas = spark.createDataFrame(pandas_df_subset, schema=schema)
spark_df_from_pandas.show(2)

+---------------+---------+-------+-----------+----+---------------+---+----+---+
|    destination|passanger|weather|temperature|time|         coupon|age| car|  Y|
+---------------+---------+-------+-----------+----+---------------+---+----+---+
|No Urgent Place|    Alone|  Sunny|         55| 2PM|Restaurant(<20)| 21|null|  1|
|No Urgent Place|Friend(s)|  Sunny|         80|10AM|   Coffee House| 21|null|  0|
+---------------+---------+-------+-----------+----+---------------+---+----+---+
only showing top 2 rows



In [ ]:
#Previously temperature was "long" data type, here we can see temperature as interger Dtype
spark_df_from_pandas.printSchema()

root
 |-- destination: string (nullable = true)
 |-- passanger: string (nullable = true)
 |-- weather: string (nullable = true)
 |-- temperature: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- coupon: string (nullable = true)
 |-- age: string (nullable = true)
 |-- car: string (nullable = true)
 |-- Y: integer (nullable = true)

